In [1]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 82.7 MB/s eta 0:00:00:00:0100:01


In [7]:
import torch

print(torch.cuda.is_available())

True


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() 
                      else "mps" if torch.backends.mps.is_available() 
                      else "cpu")

print(f"Current device: {device}")


Current device: cuda


In [8]:
import torch.nn as nn
import torch.optim as optim


In [ ]:

class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,output_size,dropout=0.2):
        super(LSTM,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,dropout=dropout,bidirectional=True)
        self.fc = nn.Linear(hidden_size,output_size)
    def forward(self,x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(x.device)
        out, _ = self.lstm(x,(h0,c0))

        last_time_step = out[:,-1,:]
        return self.fc(last_time_step)
